## Assignment 1
### Senan Gaffori, 20949022
### Ayush Bhargava, 20889700

### 1.1 - An optimization problem with Gurobipy

In [7]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("DemoExample")

x = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x")
y = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="y")

m.setObjective(x + y, GRB.MAXIMIZE)

m.addConstr(x+2*y <=4, "c1")
m.addConstr(x*4+3*y <= 12, "c2")

m.optimize()

if m.status == GRB.OPTIMAL:
    if m.status == GRB.OPTIMAL:
        print(f"Objective Value: {m.objVal}")
        print(f"x: {x.X}, y: {y.X}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9a331afa
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+30   3.250000e+30   2.000000e+00      0s
       2    3.2000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.200000000e+00
Objective Value: 3.2
x: 2.4, y: 0.8


### 1.2 - An optimization problem with CPLEX

In [ ]:
%pip install docplex

from docplex.mp.model import Model

# Create a model
mdl = Model(name='DemoDocplex')

# Variables
x = mdl.continuous_var(name='x', lb=0)
y = mdl.continuous_var(name='y', lb=0)

# Objective: Maximize x + y
mdl.maximize(x + y)

# Constraints
mdl.add_constraint(x + 2*y <= 4, 'c1')
mdl.add_constraint(x*4 + 3*y <= 12, 'c2')

# Solve
sol = mdl.solve(log_output=True)

# Output
if sol:
    print(f"Objective Value: {mdl.objective_value}")
    mdl.print_solution()
else:
    print("No solution found")

From 1.1 and 1.2 we see the final solution has optimal values of:
x = 2.4
y = 0.8
z = 3.2

## 2 - Lab Assignment

## Problem Formulation

## CFLP - Sets, Parameters and Decsion Variables

### Sets:

$$
I: \text{ Set of facilities } (f_1, f_2, f_3, f_4, f_5, f_6, f_7)
$$
$$
J: \text{ Set of customers } (c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_{10})
$$

### Parameters:
$$
f_i: \text{ Fixed cost of opening facility } i \in I
$$
$$
u_i: \text{ Capacity of facility } i \in I
$$
$$
d_j: \text{ Demand of customer } j \in J
$$
$$
c_{ij}: \text{ Transportation cost of shipping from facility } i \text{ to customer } j
$$\
$$
r: \text{ Revenue earned per unit of demand fulfilled } = \$1000
$$

### Decision Variables:
$$
y_i \in \{0,1\}: \text{ Binary variable indicating if facility } i \text{ is open}
$$
$$
x_{ij} \geq 0: \text{ Amount shipped from facility } i \text{ to customer } j
$$\
$$
z_{ij} \in \{0, 1\}: 1 \text{ if customer } j \text{ is assigned to facility } i, 0 \text{ otherwise}
$$

## Objective Function

$$
\text{Maximize } z =  \sum_{i \in I} \sum_{j \in J} r \cdot x_{ij} - \sum_{i \in I} f_i y_i - \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}
$$

## Constraints

**Each customer is served by at most one facility:**
$$
\sum_{i \in I} z_{ij} \leq 1 \quad \forall j \in J
$$

**Shipping allowed only if customer is assigned to that facility:**
$$
x_{ij} \leq d_j \cdot z_{ij} \quad \forall i \in I, \forall j \in J
$$

**A facility cannot exceed its capacity:**
$$
\sum_{j \in J} x_{ij} \leq u_i \cdot y_i \quad \forall i \in I
$$

**Customer can only be assigned to an open facility:**
$$
z_{ij} \leq y_i \quad \forall i \in I, \forall j \in J
$$

**At most 3 facilities may be opened:**
$$
\sum_{i \in I} y_i \leq 3
$$

**Variable domains:**
$$
y_i \in \{0, 1\}, \quad z_{ij} \in \{0, 1\}, \quad x_{ij} \geq 0
$$


# Code Implementation w/ Gurobi

## Import Libaries and Read CSV file w/ Customer Data

In [1]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# --- Load CSV ---
df = pd.read_csv("LabData.csv")

# --- Extract facility and customer data ---
facility_data = df[['Facility', 'Fixed Cost ($)', 'Capacity (units)']].dropna().reset_index(drop=True)
facilities = facility_data['Facility'].tolist()
fixed_cost = dict(zip(facility_data['Facility'], facility_data['Fixed Cost ($)']))
capacity = dict(zip(facility_data['Facility'], facility_data['Capacity (units)']))

customer_cols = df.columns[df.columns.get_loc('Facility.1')+1:]
customers = customer_cols.tolist()

customer_data = df[['Customer', 'Demand (units)']].dropna().reset_index(drop=True)
demand = dict(zip(customer_data['Customer'], customer_data['Demand (units)']))

distance = {}
for i, row in facility_data.iterrows():
    facility = row['Facility']
    for customer in customers:
        distance[(facility, customer)] = df.loc[i, customer]

## Build Model w/ Constraints

In [2]:
# --- Gurobi Model ---
m = Model("Lab1")

# Parameters
revenue_per_unit = 1000  # Assumed constant profit per unit delivered

# Decision variables
y = m.addVars(facilities, vtype=GRB.BINARY, name="Open")  # Open facility
x = m.addVars(facilities, customers, vtype=GRB.CONTINUOUS, name="Ship")  # Quantity shipped
z = m.addVars(facilities, customers, vtype=GRB.BINARY, name="Assign")  # Assignment

# --- Objective: Maximize profit ---
# Revenue from shipped goods - Fixed costs - Transport cost
m.setObjective(
    quicksum(revenue_per_unit * x[i, j] for i in facilities for j in customers)
    - quicksum(fixed_cost[i] * y[i] for i in facilities)
    - quicksum(distance[i, j] * x[i, j] for i in facilities for j in customers),
    GRB.MAXIMIZE
)

# --- Constraints ---

# 1. Each customer assigned to at most one facility
for j in customers:
    m.addConstr(quicksum(z[i, j] for i in facilities) <= 1, name=f"AssignOnce_{j}")

# 2. Shipping only allowed if assigned
for i in facilities:
    for j in customers:
        m.addConstr(x[i, j] <= demand[j] * z[i, j], name=f"ShipIfAssigned_{i}_{j}")

# 3. Facility capacity not exceeded
for i in facilities:
    m.addConstr(quicksum(x[i, j] for j in customers) <= capacity[i] * y[i], name=f"Capacity_{i}")

# 4. Facility must be open to serve
for i in facilities:
    for j in customers:
        m.addConstr(z[i, j] <= y[i], name=f"ServeIfOpen_{i}_{j}")

# 5. At most 3 facilities can be open
m.addConstr(quicksum(y[i] for i in facilities) <= 3, name="Max3Facilities")

# --- Solve ---
m.optimize()

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 158 rows, 147 columns and 434 nonzeros
Model fingerprint: 0xdbda41ac
Variable types: 70 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.02s
Presolved: 158 rows, 147 columns, 434 nonzeros
Variable types: 0 continuous, 147 integer (77 binary)

Root relaxation: objective 1.363050e+05, 59 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

## Print Solution

In [3]:
# --- Print results ---
if m.status == GRB.OPTIMAL:
    print(f"\nTotal Profit: {m.objVal:.2f}")
    for i in facilities:
        if y[i].x > 0.5:
            print(f"\nFacility {i} is OPEN")
            for j in customers:
                if x[i, j].x > 0:
                    print(f"  Ships {x[i, j].x:.1f} units to Customer {j}")
else:
    print("No optimal solution found.")


Total Profit: 136292.00

Facility F2 is OPEN
  Ships 22.0 units to Customer C2
  Ships 30.0 units to Customer C4
  Ships 8.0 units to Customer C8

Facility F5 is OPEN
  Ships 15.0 units to Customer C3
  Ships 27.0 units to Customer C5
  Ships 23.0 units to Customer C9

Facility F6 is OPEN
  Ships 18.0 units to Customer C1
  Ships 1.0 units to Customer C6
  Ships 15.0 units to Customer C7
  Ships 21.0 units to Customer C10


## Final Solution


**Final Solution:**

**Total Profit:** $136292.00$

**Facility F2 is OPEN**
* Ships $22.0$ units to Customer C2
* Ships $30.0$ units to Customer C4
* Ships $8.0$ units to Customer C8 (Demand not fully met)

**Facility F5 is OPEN**
* Ships $15.0$ units to Customer C3 (Demand not fully met)
* Ships $27.0$ units to Customer C5
* Ships $23.0$ units to Customer C9

**Facility F6 is OPEN**
* Ships $18.0$ units to Customer C1
* Ships $1.0$ unit to Customer C6 (Demand not fully met)
* Ships $15.0$ units to Customer C7
* Ships $21.0$ units to Customer C10

## Takeaways

### In order to satsify the objective function and maximize profit for the company, demand could not be fully met for customers 3, 6, and 8.

## GenAI Disclosure

GenAI and LLMs were used to format the text in the jupyter notebook to be a user friendly reading form.
